In [1]:
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import cpi
import pymongo
from tqdm import tqdm
from scipy import stats
import math
import numpy as np
import pandas as pd
import json

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["imdb"]
imdb = db['imdb_cleaned']

In [3]:
num_movies = imdb.count_documents({})
num_movies

8181

In [4]:
def getMovie(imdb_id):
    for x in imdb.find({'imdb_id': imdb_id}):
        return x
    return None

In [7]:
prod_features = {}
with open('SavedFeatures/prod_co_features.json', 'r') as f:
    prod_features = json.load(f)

In [8]:
len(prod_features.keys())

7422

In [11]:
prod = prod_features['Universal Pictures']

In [12]:
prod

{'name': 'Universal Pictures',
 'year-wise-performance': {'1967': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1968': {'num_movies': 1,
   'sum_rating': 6.5,
   'sum_revenue': 23129373,
   'sum_budget': 11155646,
   'num_raters': 15925},
  '1969': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1970': {'num_movies': 1,
   'sum_rating': 7.0,
   'sum_revenue': 33685452,
   'sum_budget': 16675966,
   'num_raters': 24298},
  '1971': {'num_movies': 1,
   'sum_rating': 7.0,
   'sum_revenue': 67738187,
   'sum_budget': 4633036,
   'num_raters': 26215},
  '1972': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1973': {'num_movies': 2,
   'sum_rating': 15.700000000000001,
   'sum_revenue': 1579681639,
   'sum_budget': 36589113,
   'num_raters': 328026},
  '1974': {'num_movies': 2,
   'sum_rating': 12.7,
   'sum_revenue': 45762558

In [13]:
rolling_year = 5
present = 2020
first_movie_year = 1967

for year in rdj['year-wise-performance']:
    if(rdj['year-wise-performance'][year]['num_movies'] != 0):
        first_movie_year = int(year)
        break
first_movie_year

1968

In [14]:
def getValues(st_year, nd_year, entity):
    info = {}
    info['total_revenue'] = 0
    info['total_budget'] = 0
    info['num_movies'] = 0
    info['num_raters'] = 0
    for year in range(st_year, nd_year+1):
        year = str(year)
        info['total_revenue']   += entity['year-wise-performance'][year]['sum_revenue']
        info['total_budget']    += entity['year-wise-performance'][year]['sum_budget']
        info['num_movies']      += entity['year-wise-performance'][year]['num_movies']
        info['num_raters']      += entity['year-wise-performance'][year]['num_raters']
    return info

In [15]:
getValues(first_movie_year, present, rdj)

{'total_revenue': 92509059470,
 'total_budget': 27927229094,
 'num_movies': 456,
 'num_raters': 58916715}

In [16]:
year_label = []
revenue_arr = []
budget_arr = []
num_movie_arr = []

for year in range(first_movie_year+rolling_year-1, present):
    st_year = year - rolling_year + 1
    nd_year = year
    info = getValues(st_year, nd_year, rdj)
    year_label.append('{} - {}'.format(st_year, nd_year))

    if(info['num_movies'] != 0):
        revenue_arr.append(info['total_revenue']/info['num_movies'])
        budget_arr.append(info['total_budget']/info['num_movies'])
        num_movie_arr.append(info['num_movies'])
    else:
        revenue_arr.append(0)
        budget_arr.append(0)
        num_movie_arr.append(0) 

In [17]:
def plotFigure(xrr, yrr, zrr):
    fig = make_subplots()
    fig.add_trace(
        go.Scatter(
            x=xrr, 
            y=yrr, 
            name="Revenue",
            mode='lines+markers',
            line_color='rgba(102, 0, 204, .8)'
        ),
    )
    
    fig.add_trace(
        go.Scatter(
            x=xrr, 
            y=zrr, 
            name="Budget",
            mode='lines+markers',
            line_color='rgba(153, 0, 51, .8)'
        ),
    )
    return fig

In [18]:
fig = plotFigure(year_label, revenue_arr, budget_arr)
fig.show()

# Star Power -- KS Test

In [19]:
def getEntityID(entity):
    # print(entity)
    id = entity['url'].split('?')[0].split('/')[-2]
    return id

In [20]:
def getStarInfo(star_id):
    star_info = star_features[star_id]
    return star_info['total_movies'], star_info['total_raters']

In [21]:
movie = getMovie('tt0371746')
# movie

In [22]:
def getStarInfoBefore(star_id, year):
    return getValues(1967, year-1, star_features[star_id])

In [23]:
getStarInfo('Universal Pictures')

(456, 58916715)

In [24]:
getStarInfoBefore('Universal Pictures', 2008)

{'total_revenue': 57446804840,
 'total_budget': 18690842056,
 'num_movies': 318,
 'num_raters': 36054100}

In [29]:
th_num_movie = 20
th_avg_raters = 500000

def getMovieCriteria(movie, printLog=False):
    cast_ensamble = movie['cast_and_character']
    if('Production Co' not in movie['details']):
        if(printLog == True):
            print(movie['imdb_id'], "could ont find cast_ensamble")
    else:
        prod_arr = movie['details']['Production Co']
        if(type(prod_arr) is not list):
            prod_arr = [prod_arr]
        for cast in prod_arr:
            star = cast
            star_id = star
            # num_movie, total_raters = getStarInfo(star_id)
            info = getStarInfoBefore(star_id, int(movie['cleaned_year']))
            num_movie = info['num_movies']
            total_raters = info['num_raters']
            avg_raters = 0
            if(num_movie != 0):
                avg_raters = total_raters/num_movie
            # print(star_id, num_movie, th_num_movie, total_raters, total_raters/num_movie, th_avg_raters,  star['name'])
            # print(num_movie > th_num_movie, avg_raters > th_avg_raters)
            if(num_movie > th_num_movie or avg_raters > th_avg_raters):
                return True
    return False

In [30]:
getMovieCriteria(movie)

True

In [31]:
has_star = []
no_star = []

for movie in imdb.find():
    st = getMovieCriteria(movie)
    revenue = movie['cleaned_Revenue']
    if(st == True):
        has_star.append((movie['imdb_id'], movie['name'], revenue))
    else:
        no_star.append((movie['imdb_id'], movie['name'], revenue))

In [32]:
len(has_star), len(no_star)

(3487, 4694)

In [33]:
has_star_revenue = []
no_star_revenue = []

for movie in has_star:
    has_star_revenue.append(movie[2])

for movie in no_star:
    no_star_revenue.append(movie[2])

In [34]:
len(has_star_revenue), len(no_star_revenue)

(3487, 4694)

In [35]:
from scipy.stats import ks_2samp
import numpy as np

has_star_revenue = np.array(has_star_revenue)
no_star_revenue = np.array(no_star_revenue)

In [36]:
ks_2samp(has_star_revenue, no_star_revenue)

KstestResult(statistic=0.4834090075145507, pvalue=5.276495262940273e-08)

In [37]:
has_star_revenue.mean(), no_star_revenue.mean()

(173658993.92801836, 46583439.662121855)

In [38]:
has_star_revenue.mean()-no_star_revenue.mean()

127075554.2658965